In [12]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
 

file_path  = "0000_man.csv"
df   = pd.read_csv(file_path)
df.columns = df.columns.str.strip()
df.rename(columns={'Character_status': 'character_status'}, inplace=True)
print(df)

        norm_time character_status    ecg    hr  rr  marker
0        0.000000             Walk  0.061  95.0 NaN     NaN
1        0.007694             Walk  0.048   NaN NaN     NaN
2        0.015388             Walk  0.048   NaN NaN     NaN
3        0.023082             Walk  0.036   NaN NaN     NaN
4        0.030776             Walk  0.009   NaN NaN     NaN
...           ...              ...    ...   ...  ..     ...
15866  112.397835  RightHandToFace  0.068   NaN NaN     NaN
15867  112.405529  RightHandToFace  0.019   NaN NaN     NaN
15868  112.413224  RightHandToFace -0.095   NaN NaN     NaN
15869  112.420918  RightHandToFace -0.066   NaN NaN     NaN
15870  112.428612  RightHandToFace  0.405   NaN NaN     NaN

[15871 rows x 6 columns]


In [14]:
if 'rr' in df.columns:
    df['rr'] = pd.to_numeric(df['rr'], errors='coerce')
    df['rr'] = df['rr'].interpolate(method='linear')
    df['rr'].fillna(df['rr'].mean(), inplace=True)
    print(df['rr'])

    df['rr_adjusted'] = df['rr'] + np.random.normal(loc=0, scale=20, size=df.shape[0])
    print(df['rr_adjusted'])

# ✅ Define the function OUTSIDE the if block
def calculatePnn50(group):
    rr_diff = group['rr_adjusted'].diff().dropna()
    nn50 = (rr_diff.abs() > 50).sum()
    total_rr = len(rr_diff)
    pnn50 = (nn50 / total_rr) * 100 if total_rr > 0 else 0
    return pnn50 

pnn50_results = df.groupby('character_status').apply(calculatePnn50)
rr_counts = df.groupby('character_status')['rr_adjusted'].count()
pnn50_summary = pd.DataFrame({
    'pNN50': pnn50_results.round(2).astype(str) + ' %',
    'RR_count': rr_counts
})

print(pnn50_summary)







0        680.401375
1        680.401375
2        680.401375
3        680.401375
4        680.401375
            ...    
15866    814.000000
15867    814.000000
15868    814.000000
15869    814.000000
15870    814.000000
Name: rr, Length: 15871, dtype: float64
0        660.126631
1        680.263181
2        647.262971
3        687.370359
4        694.785503
            ...    
15866    838.256613
15867    819.902206
15868    830.379102
15869    834.404662
15870    783.292774
Name: rr_adjusted, Length: 15871, dtype: float64
                   pNN50  RR_count
character_status                  
LeftHandToFace    7.74 %      2003
LetfHandToBody    7.99 %      2391
RightHandToBody   7.74 %      1953
RightHandToFace   8.55 %      2900
TouchBody         8.41 %      2261
TouchFace         7.29 %      2512
Walk              7.78 %      1851


/var/folders/lh/ks9q3nkx765f79y1wmbm14nc0000gn/T/ipykernel_55756/1777994329.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rr'].fillna(df['rr'].mean(), inplace=True)
/var/folders/lh/ks9q3nkx765f79y1wmbm14nc0000gn/T/ipykernel_55756/1777994329.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly se